In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale
from sklearn import metrics as mt # mean_squared_error, confusion_matrix, roc_auc_score
from sklearn.neighbors import NearestNeighbors

Imports the cleaned data set (from HealthCost_data_cleaning notebook)

In [ ]:
df=pd.read_csv('data/IPPS_Data_Clean.csv', \
               dtype={'provider_id':str,'provider_zip_code':str, 'drg_id':str,'total_discharges':float})

#df['provider_city_state'] = df.provider_city + ', ' + df.provider_state
df.head(3)

GPS locations of every unique provider.

In [ ]:
dfprovgeo=pd.read_csv('data/Providers_Geocode.csv',usecols=['provider_id','lat','lng'],dtype={'provider_id':str})
dfprovgeo.head(3)

Percentage of uninsured by state (2013)

In [ ]:
dfins = pd.read_csv('data/InsuranceCoverage_Adults19-64.csv',usecols=['Location','Uninsured'])#,encoding='Latin')
dfins=dfins[1:].reset_index(drop=True)#.head()

# Dataframe that lists states and their abbreviations for mapping
abbrev = pd.read_csv('data/states.csv').rename(columns={'State':'Location','Abbreviation':'provider_state'})
dfins = dfins.merge(abbrev,on ='Location')[['provider_state','Uninsured']]
dfins.head(3)

Population by state dataframe (Census.gov)

In [ ]:
dfstates=pd.read_csv('data/NST-EST2015-alldata.csv',usecols=['NAME','POPESTIMATE2011'])#,dtype={'POPESTIMATE2011':float})
dfstates = dfstates[5:56].reset_index(drop=True)

# Dataframe that lists states and their abbreviations for mapping
abbrev = pd.read_csv('data/states.csv').rename(columns={'State':'NAME','Abbreviation':'provider_state'})

dfstates = dfstates.merge(abbrev,on ='NAME')
dfstates = dfstates[['provider_state','POPESTIMATE2011']]#,'RBIRTH2011','RDEATH2011']]
dfstates = dfstates.rename(columns={'POPESTIMATE2011':'state_population2011'})

dfstates.head(3)

Population by zip code dataframe (http://notnullhypothesis.com/2010-census-data-by-zip-code-population/)

In [ ]:
#dfzip = pd.read_csv('2010CensusPopulationData.csv',usecols=['Zip','Population'],dtype={'Zip':str})
dfzip = pd.read_csv('data/Zipcode-ZCTA-Population-Density-And-Area-Unsorted.csv', \
                   dtype={'Zip':str},usecols=['Zip','2010 Population','Density Per Sq Mile'])
dfzip.Zip = dfzip.Zip.str.zfill(5)
dfzip = dfzip.rename(columns = {'Zip':'provider_zip_code','2010 Population':'zip_population2010', \
                               'Density Per Sq Mile':'zip_pop2010_density'})
dfzip.head()

Median income by zipcode (http://www.psc.isr.umich.edu/dis/census/Features/tract2zip/)

In [ ]:
dfincome = pd.read_csv('data/MedianZIP-3.csv',dtype={'Zip':str},usecols=['Zip','Median'])

dfincome.Zip = dfincome.Zip.str.zfill(5)
dfincome.Median = dfincome.Median.str.replace(',','').astype(int)
dfincome = dfincome.rename(columns={'Zip':'provider_zip_code','Median':'zip_median_income'})

dfincome.head()

Combines all into one dataframe and selects the relevant columns

In [ ]:
#df['provider_city_state'] = df.provider_city + ', ' + df.provider_state

dftot = df.merge(dfprovgeo, on='provider_id')
#dftot = dftot.merge(dfins,on='provider_state')
#dftot = dftot.merge(dfstates, on='provider_state')
#dftot = dftot.merge(dfzip, on='provider_zip_code',how='left')
#dftot = dftot.merge(dfcities, on=['provider_city_state'])
#dftot = dftot.merge(dfincome, on='provider_zip_code')
#dftot = dftot.drop([list(dftot.columns)[1]]+list(dftot.columns)[3:5]+list(dftot.columns)[11:13],axis=1)
#dftot.head()
#dftot.provider_id.nunique()
print(len(dftot))
dftot.head(3)

Calculates the national median cost for each procedure

In [ ]:
natmed = pd.DataFrame(dftot.groupby('drg_id',sort=False).average_covered_charges.median()).reset_index()
natmed = natmed.rename(columns={'average_covered_charges':'median_covered_charges'})
natmed.head()

Adds a column that is the fractional difference in total charges from the national median (for that particular procedure) and a column classifying whether the fractional difference is above (True) or below (False) the national median.

In [ ]:
procedures=dftot.drg_id.unique()

for i in procedures:    
    med = natmed.loc[natmed.drg_id == i,'median_covered_charges'].iloc[0]
    dftot.loc[dftot.drg_id == i,'frac_diff'] = dftot.loc[dftot.drg_id == i,'average_covered_charges']/med - 1.0

dftot['frac_diff_class'] = dftot.frac_diff >= 0.0

dftot.head(3)

In [ ]:
dftot.groupby('frac_diff_class').size()

For each provider, calculates the number of other providers within a 40 mile radius (Takes around 3.5 minutes to run)

In [ ]:
from geopy.distance import vincenty
import time

In [ ]:
%%time
dfprovgeo['n_hosp'] = 0.0

pts = [dfprovgeo.lat,dfprovgeo.lng]
pts = list(list(x) for x in zip(*pts))
i=0

for index, row in dfprovgeo[['lat','lng']].iterrows():
    count = 0
    pt1 = [row.lat,row.lng]
    
    for pt in pts:
        dist = vincenty(pt1, pt).miles
        if dist < 40 and dist > 0:
            count +=1
            
    dfprovgeo.loc[index,'n_hosp'] = count
    i+=1
    if i % 1000 == 0: print(i)

In [ ]:
dfprovgeo.head()

In [ ]:
dftot = dftot.merge(dfprovgeo[['provider_id','n_hosp']], on='provider_id')

In [ ]:
dftot.head(3)

Random Forest Classifier using entire dataset trying to predict whether a given procedure at a provider will be above or below the national median.

Selects the variables for X and y for the random forest classifier.

In [ ]:
inx = ['lat','lng']#'provider_city']#,'zip_pop2010_density','zip_population2010','zip_median_income']#,'provider_id']#['lat','lng']#
dfX = dftot.loc[:,inx]#zip_population2010']]
dfY = dftot.loc[:,'frac_diff_class']

dfX.head()

In [ ]:
# dfX = pd.get_dummies(dfX, columns=['drg_id'], dummy_na=True)
# dfX.head()

Does the test-train split and runs the RFC on the training set. It then calculates the R^2 score using the model to predict the test set.

In [ ]:
rf = RandomForestClassifier(n_jobs=-1)
#rf.n_estimators=10

X_train, X_test, y_train, y_test = train_test_split(dfX, dfY, train_size=2./3)

rf.fit(X_train,y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

rf.score(X_test,y_test)

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(2)
    tick_label = ['f < 0','f > 0']
    plt.xticks(tick_marks, tick_label, rotation=45)
    plt.yticks(tick_marks, tick_label)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

Plots the confusion matrix comparing the predicted label to the true label.

In [ ]:
cm = mt.confusion_matrix(y_test, y_test_pred)

print('Confusion matrix')
print(cm)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)


plt.figure()
plot_confusion_matrix(cm,title='Confusion matrix, without normalization')
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

Plots the ROC curve and calculates the AUC score for the RFR model.

In [ ]:
y_score = rf.predict_proba(X_test)[:,1]

print('AUC Score =',mt.roc_auc_score(y_test, y_score))
fpr, tpr, thresholds = mt.roc_curve(y_test,y_score)
plt.figure()
plt.plot(fpr,tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate');

Adding in other variables such as the population and median income by zip code seemed to have no significant effect. I also calculated the number of other providers within 40 miles of each provider, but adding that variable had no effect as well.

I also found that I get the same results with just the provider_id tag as the input X using pandas’ get_dummies to convert the variable to an indicator variable rather than numerical value. This makes me think that the GPS coordinates are essentially acting as an indicator for the specific provider rather than giving any useful information as far as relative position to other hospitals.